In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/root/.conda/envs/social/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = '/root/share/model_repos/internlm2-chat-20b'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# Set `torch_dtype=torch.float16` to load model in float16, otherwise it will be loaded as float32 and cause OOM Error.
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, trust_remote_code=True).cuda()
model = model.eval()

Loading checkpoint shards:   0%|          | 0/21 [00:00<?, ?it/s]/root/.conda/envs/social/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 21/21 [00:51<00:00,  2.44s/it]


In [7]:
prompt = """
Answer the following questions as best you can. You have access to the following tools:

google_search: Call this tool to interact with the 谷歌搜索 API. What is the 谷歌搜索 API useful for? 谷歌搜索是一个通用搜索引擎，可用于访问互联网、查询百科知识、了解时事新闻等。 Parameters: [{'name': 'search_query', 'description': '搜索关键词或短语', 'required': True, 'schema': {'type': 'string'}}] Format the arguments as a JSON object.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [google_search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can be repeated zero or more times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 特朗普哪一年出生的？
"""

response, history = model.chat(tokenizer, prompt, history=[], temperature=0.1)
print(response)

Thought: 为了回答这个问题，我将使用谷歌搜索API来查找特朗普的出生日期。
Action: google_search
Action Input: {"search_query": "特朗普 出生日期"} 


In [1]:
tet = '''第一次： Thought: 你好，请问有什么我可以帮助你的吗？
Action: google_search
Action Input: {'search_query': '你好'}'''

def parse_latest_plugin_call(text):
    plugin_name, plugin_args = '', ''
    i = text.rfind('\nAction:')
    j = text.rfind('\nAction Input:')
    k = text.rfind('\nObservation:')
    if 0 <= i < j:  # If the text has `Action` and `Action input`,
        if k < j:  # but does not contain `Observation`,
            # then it is likely that `Observation` is ommited by the LLM,
            # because the output text may have discarded the stop word.
            text = text.rstrip() + '\nObservation:'  # Add it back.
        k = text.rfind('\nObservation:')
        plugin_name = text[i + len('\nAction:') : j].strip()
        plugin_args = text[j + len('\nAction Input:') : k].strip()
        text = text[:k]
    return plugin_name, plugin_args

print(parse_latest_plugin_call(tet))

('google_search', "{'search_query': '你好'}")


In [4]:
import json5

In [5]:
json5.loads(parse_latest_plugin_call(tet)[1])

{'search_query': '你好'}